In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

from scipy import interpolate
from scipy.stats import spearmanr

In [2]:
file_path = './data/rainfall_train.csv'
rainfall = pd.read_csv(file_path)
rainfall = rainfall.iloc[:, 1:]
rainfall.columns = rainfall.columns.str.replace('rainfall_train.', '')
rainfall

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
0,A,5,1,9,STN001,3,A,5,1,12,...,0,0,0,0,0,0,0,0,0.0,0
1,A,5,1,9,STN001,6,A,5,1,15,...,0,0,0,0,0,0,0,0,0.0,0
2,A,5,1,9,STN001,9,A,5,1,18,...,0,0,0,0,0,0,0,0,0.0,0
3,A,5,1,9,STN001,12,A,5,1,21,...,0,0,0,0,0,0,0,0,0.0,0
4,A,5,1,9,STN001,15,A,5,2,0,...,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457247,C,9,30,21,STN020,228,C,10,10,9,...,7,2,2,0,0,0,0,0,0.5,3
1457248,C,9,30,21,STN020,231,C,10,10,12,...,9,5,3,1,0,0,0,0,0.0,0
1457249,C,9,30,21,STN020,234,C,10,10,15,...,9,5,3,1,0,0,0,0,1.9,4
1457250,C,9,30,21,STN020,237,C,10,10,18,...,6,4,2,2,0,0,0,0,1.9,4


# 결측치

In [8]:
### 전체 데이터 속 통으로 관측되지 않은 날
temp = rainfall.drop_duplicates(subset=['fc_year','fc_month','fc_day', 'fc_hour'], keep='last')
temp[(temp['fc_year'] == 'B') & (temp['fc_month'] == 5)].iloc[21:27]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
946439,B,5,11,21,STN020,240,B,5,21,21,...,23,17,12,10,7,1,0,0,0.0,0
946519,B,5,12,9,STN020,240,B,5,22,9,...,5,2,2,2,2,2,1,0,0.0,0
946599,B,5,12,21,STN020,240,B,5,22,21,...,8,6,5,4,0,0,0,0,0.0,0
946679,B,5,16,9,STN020,240,B,5,26,9,...,0,0,0,0,0,0,0,0,0.0,0
946759,B,5,16,21,STN020,240,B,5,26,21,...,2,2,2,2,2,0,0,0,0.0,0
946839,B,5,17,9,STN020,240,B,5,27,9,...,5,4,0,0,0,0,0,0,0.0,0


In [6]:
### 중간에 관측이 빠진 날짜 확인
idx = rainfall[rainfall['dh'] == 3].index #18239
idx2 = rainfall[rainfall['dh'] == 240].index #18159

### 240으로 80번 예측되지 않은 날이 있음을 확인 -> dic으로 리스트 추출
dic = {}

for i, j in enumerate(idx):
    if i == 0:
        continue
    
    t = rainfall.iloc[idx[i-1]:j]
    
    if len(t) != 80:
        tail = t.iloc[-1, 5]
        s1 = set(list(range(3,tail+1,3))) - set(t['dh'].values)
        
        if len(s1) > 0:
            a = t.iloc[0][:5]
            dic[f'{a[0]}_{a[1]}_{a[2]}_{a[3]}_{a[4]}'] = s1
        
dic

{'A_6_25_9_STN001': {207, 210},
 'A_9_6_9_STN001': {165, 168},
 'A_6_25_9_STN002': {207, 210},
 'A_9_6_9_STN002': {165, 168},
 'A_6_25_9_STN003': {207, 210},
 'A_9_6_9_STN003': {165, 168},
 'A_6_25_9_STN004': {207, 210},
 'A_9_6_9_STN004': {165, 168},
 'A_5_1_9_STN005': {231, 234, 237, 240},
 'A_5_16_21_STN005': {75,
  78,
  81,
  84,
  87,
  90,
  93,
  96,
  99,
  102,
  105,
  108,
  111,
  114,
  117,
  120,
  123,
  126,
  129,
  132,
  135,
  138,
  141,
  144,
  147,
  150,
  153,
  156,
  159,
  162,
  165,
  168,
  171,
  174,
  177,
  180,
  183,
  186,
  189,
  192,
  195,
  198,
  201,
  204,
  207,
  210,
  213,
  216,
  219,
  222,
  225,
  228,
  231,
  234,
  237,
  240},
 'A_6_4_21_STN005': {207,
  210,
  213,
  216,
  219,
  222,
  225,
  228,
  231,
  234,
  237,
  240},
 'A_6_15_9_STN005': {72,
  75,
  78,
  81,
  84,
  87,
  90,
  93,
  96,
  99,
  102,
  105,
  108,
  111,
  114,
  117,
  120,
  123,
  126,
  129,
  132,
  135,
  138,
  141,
  144,
  147,
  150,
 

In [7]:
# 18239일 중 160일이 빠진 관측시간이 있음
len(dic.keys())

160

-------

In [30]:
a,b,c,d,e = 'C_9_17_9_STN020'.split('_')
temp2 = rainfall[(rainfall['fc_year'] == a) & (rainfall['fc_month'] == int(b)) & (rainfall['fc_day'] == int(c)) & (rainfall['fc_hour'] == int(d)) & (rainfall['stn4contest'] == e)]
rainfall[(rainfall['fc_year'] == a) & (rainfall['fc_month'] == int(b)) & (rainfall['fc_day'] == int(c)) & (rainfall['fc_hour'] == int(d)) & (rainfall['stn4contest'] == e)]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
1455046,C,9,17,9,STN020,3,C,9,17,12,...,2,0,0,0,0,0,0,0,0.0,0
1455047,C,9,17,9,STN020,6,C,9,17,15,...,29,3,0,0,0,0,0,0,0.0,0
1455048,C,9,17,9,STN020,9,C,9,17,18,...,3,0,0,0,0,0,0,0,0.0,0
1455049,C,9,17,9,STN020,12,C,9,17,21,...,28,18,6,5,2,1,0,0,0.0,0
1455050,C,9,17,9,STN020,15,C,9,18,0,...,75,61,51,34,6,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455117,C,9,17,9,STN020,228,C,9,26,21,...,12,9,5,4,4,2,0,0,0.0,0
1455118,C,9,17,9,STN020,231,C,9,27,0,...,17,11,10,8,6,3,0,0,0.0,0
1455119,C,9,17,9,STN020,234,C,9,27,3,...,17,11,10,8,6,3,0,0,0.0,0
1455120,C,9,17,9,STN020,237,C,9,27,6,...,14,12,8,7,5,3,0,0,0.0,0


# Rolling / Shift

In [ ]:
col = [f'v0{i}' for i in range(1,10)]

# drop index list
t_idx = list(idx[:])

for x in range(1,15):
    t_idx += list(idx+x)

# roll
for val in col:
    temp = rainfall[[val]]
    for x in range(1, 14+1):
        temp[f'shift_{x}'] = temp[val].shift(x)
        temp[f'roll_{x}'] = temp[val].rolling(x, min_periods=1).mean()
    temp = pd.concat([temp, rainfall['class_interval']], axis=1)
    temp.drop(index = t_idx, inplace=True)

    # 상관계수
    print(val, ' : ----------------------------------------------------------------')
    tmp = temp.corr()['class_interval'][:]
    print(tmp.sort_values(ascending=False)[:5])

# 전처리

## 시간 결측치 채우기

In [5]:
def insert_row(idx, df, df_insert):
    return pd.concat([df.loc[:idx, :], pd.DataFrame([df_insert], columns=df.columns), df.loc[idx+1:, :]], axis=0).reset_index(drop = True)


### 전처리용 df생성
df_t = rainfall.copy()

### 시간 결측치 채우기
for key in dic.keys():
    a,b,c,d,e = key.split('_')
    lis = sorted(list(dic[key]))
    
    for val in lis:
        temp2 = df_t[(df_t['fc_year'] == a) & (df_t['fc_month'] == int(b)) & (df_t['fc_day'] == int(c)) & (df_t['fc_hour'] == int(d)) & (df_t['stn4contest'] == e)]
        val2 = val-3
        idx3 = temp2[temp2['dh'] == val2].index[0]
        
        # pred time
        f,g,h,i = temp2.loc[idx3, ['ef_year', 'ef_month', 'ef_day', 'ef_hour']].values

        date_time = datetime(2024, g,h,i, 0)
        new_date_time = date_time + timedelta(hours=3)

        df_t = insert_row(idx3, df_t, [a,int(b),int(c),int(d),e,val, f, new_date_time.month, new_date_time.day, new_date_time.hour]+[np.nan]*11) #row_insert
        
# df_t.to_csv('./data/rainfall_train_v1.csv')
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457432 entries, 0 to 1457431
Data columns (total 21 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fc_year         1457432 non-null  object 
 1   fc_month        1457432 non-null  int64  
 2   fc_day          1457432 non-null  int64  
 3   fc_hour         1457432 non-null  int64  
 4   stn4contest     1457432 non-null  object 
 5   dh              1457432 non-null  int64  
 6   ef_year         1457432 non-null  object 
 7   ef_month        1457432 non-null  int64  
 8   ef_day          1457432 non-null  int64  
 9   ef_hour         1457432 non-null  int64  
 10  v01             1457252 non-null  float64
 11  v02             1457252 non-null  float64
 12  v03             1457252 non-null  float64
 13  v04             1457252 non-null  float64
 14  v05             1457252 non-null  float64
 15  v06             1457252 non-null  float64
 16  v07             1457252 non-null  fl

In [5]:
df_t.iloc[8865:8865+15]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
8865,A,6,25,9,STN001,198,A,7,3,15,...,32.0,20.0,14.0,12.0,4.0,0.0,0.0,0.0,0.0,0.0
8866,A,6,25,9,STN001,201,A,7,3,18,...,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0,0.0,0.0
8867,A,6,25,9,STN001,204,A,7,3,21,...,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0,0.0,0.0
8868,A,6,25,9,STN001,207,A,7,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8869,A,6,25,9,STN001,210,A,7,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8870,A,6,25,9,STN001,213,A,7,4,6,...,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0,0.0,0.0
8871,A,6,25,9,STN001,216,A,7,4,9,...,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0,0.0,0.0
8872,A,6,25,9,STN001,219,A,7,4,12,...,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0,0.0,0.0
8873,A,6,25,9,STN001,222,A,7,4,15,...,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0,0.0,0.0
8874,A,6,25,9,STN001,225,A,7,4,18,...,47.0,40.0,35.0,28.0,17.0,6.0,1.0,1.0,0.0,0.0


In [6]:
df_t[pd.isna(df_t['v05'])]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
8868,A,6,25,9,STN001,207,A,7,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8869,A,6,25,9,STN001,210,A,7,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20534,A,9,6,9,STN001,165,A,9,13,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20535,A,9,6,9,STN001,168,A,9,13,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57348,A,6,25,9,STN002,207,A,7,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433343,C,5,3,9,STN020,126,C,5,8,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455278,C,9,17,9,STN020,171,C,9,24,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455279,C,9,17,9,STN020,174,C,9,24,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455290,C,9,17,9,STN020,207,C,9,26,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 선형보간

In [6]:
col = ['ef_year','ef_month','ef_day','ef_hour'] + [f'v0{i}' for i in range(1,10)]
temp = df_t[col]

temp['ef_year'].replace({'A' : 2021, 'B':2022, 'C':2023}, inplace=True) #임시값
temp['datetime'] = temp.apply(lambda row: datetime(int(row['ef_year']), int(row['ef_month']), int(row['ef_day']), int(row['ef_hour'])), axis=1)
display(temp[:5])

temp.set_index('datetime', inplace=True)
temp = temp.interpolate(method='time', kind='quadratic')
temp.iloc[8865:8865+10]

,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,datetime
0,2021,5,1,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-05-01 12:00:00
1,2021,5,1,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-05-01 15:00:00
2,2021,5,1,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-05-01 18:00:00
3,2021,5,1,21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-05-01 21:00:00
4,2021,5,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-05-02 00:00:00


,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09
datetime,,,,,,,,,,,,,
2021-07-03 15:00:00,2021,7,3,15,41.0,32.0,20.0,14.0,12.0,4.0,0.0,0.0,0.0
2021-07-03 18:00:00,2021,7,3,18,38.0,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0
2021-07-03 21:00:00,2021,7,3,21,38.0,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0
2021-07-04 00:00:00,2021,7,4,0,8.0,7.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
2021-07-04 03:00:00,2021,7,4,3,24.0,21.0,20.0,17.0,14.0,9.0,5.0,1.0,0.0
2021-07-04 06:00:00,2021,7,4,6,44.0,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0
2021-07-04 09:00:00,2021,7,4,9,44.0,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0
2021-07-04 12:00:00,2021,7,4,12,60.0,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0
2021-07-04 15:00:00,2021,7,4,15,60.0,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0


In [7]:
### 덮어씌우기
# v01~v09
col = [f'v0{i}' for i in range(1,10)]
temp.reset_index(drop=True, inplace=True)
df_t[col] = temp[col]

# 실제 강수량
df_t[['vv', 'class_interval']] = df_t[['vv', 'class_interval']].fillna(method = 'ffill')

df_t.iloc[8865:8865+10]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
8865,A,6,25,9,STN001,198,A,7,3,15,...,32.0,20.0,14.0,12.0,4.0,0.0,0.0,0.0,0.0,0.0
8866,A,6,25,9,STN001,201,A,7,3,18,...,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0,0.0,0.0
8867,A,6,25,9,STN001,204,A,7,3,21,...,31.0,23.0,18.0,11.0,8.0,2.0,0.0,0.0,0.0,0.0
8868,A,6,25,9,STN001,207,A,7,4,0,...,7.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
8869,A,6,25,9,STN001,210,A,7,4,3,...,21.0,20.0,17.0,14.0,9.0,5.0,1.0,0.0,0.0,0.0
8870,A,6,25,9,STN001,213,A,7,4,6,...,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0,0.0,0.0
8871,A,6,25,9,STN001,216,A,7,4,9,...,42.0,34.0,28.0,21.0,15.0,10.0,6.0,4.0,0.0,0.0
8872,A,6,25,9,STN001,219,A,7,4,12,...,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0,0.0,0.0
8873,A,6,25,9,STN001,222,A,7,4,15,...,52.0,39.0,31.0,21.0,6.0,5.0,2.0,0.0,0.0,0.0
8874,A,6,25,9,STN001,225,A,7,4,18,...,47.0,40.0,35.0,28.0,17.0,6.0,1.0,1.0,0.0,0.0


## 파생변수

In [8]:
### 시간형
col = ['fc_year','fc_month','fc_day','fc_hour'] + ['ef_year','ef_month','ef_day','ef_hour']
temp = df_t[col]

temp[['fc_year', 'ef_year']] = temp[['fc_year', 'ef_year']].replace({'A': 2021, 'B': 2022, 'C': 2023}) #임시값
temp['datetime'] = temp.apply(lambda row: datetime(int(row['ef_year']), int(row['ef_month']), int(row['ef_day']), int(row['ef_hour'])), axis=1)

temp['quarter'] = temp['datetime'].dt.quarter
temp['timestamp'] = temp['datetime'].apply(lambda x: x.timestamp())
temp['ef_day_y'] = temp['datetime'].dt.dayofyear


temp['datetime2'] = temp.apply(lambda row: datetime(int(row['fc_year']), int(row['fc_month']), int(row['fc_day']), int(row['fc_hour'])), axis=1)
temp['fc_day_y'] = temp['datetime2'].dt.dayofyear

display(temp.iloc[8865:8865+10])

### 변수 추가하기
df_t = pd.concat([df_t, temp[['quarter', 'timestamp', 'ef_day_y', 'fc_day_y']]], axis=1)

,fc_year,fc_month,fc_day,fc_hour,ef_year,ef_month,ef_day,ef_hour,datetime,quarter,timestamp,ef_day_y,datetime2,fc_day_y
8865,2021,6,25,9,2021,7,3,15,2021-07-03 15:00:00,3,1.625324e+09,184,2021-06-25 09:00:00,176
8866,2021,6,25,9,2021,7,3,18,2021-07-03 18:00:00,3,1.625335e+09,184,2021-06-25 09:00:00,176
8867,2021,6,25,9,2021,7,3,21,2021-07-03 21:00:00,3,1.625346e+09,184,2021-06-25 09:00:00,176
8868,2021,6,25,9,2021,7,4,0,2021-07-04 00:00:00,3,1.625357e+09,185,2021-06-25 09:00:00,176
8869,2021,6,25,9,2021,7,4,3,2021-07-04 03:00:00,3,1.625368e+09,185,2021-06-25 09:00:00,176
8870,2021,6,25,9,2021,7,4,6,2021-07-04 06:00:00,3,1.625378e+09,185,2021-06-25 09:00:00,176
8871,2021,6,25,9,2021,7,4,9,2021-07-04 09:00:00,3,1.625389e+09,185,2021-06-25 09:00:00,176
8872,2021,6,25,9,2021,7,4,12,2021-07-04 12:00:00,3,1.625400e+09,185,2021-06-25 09:00:00,176
8873,2021,6,25,9,2021,7,4,15,2021-07-04 15:00:00,3,1.625411e+09,185,2021-06-25 09:00:00,176
8874,2021,6,25,9,2021,7,4,18,2021-07-04 18:00:00,3,1.625422e+09,185,2021-06-25 09:00:00,176


In [9]:
### 장소 str -> label
df_t['stn4contest'] = df_t['stn4contest'].str[-2:].astype(int)

# 확인
df_t.iloc[8865:8865+10]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v06,v07,v08,v09,vv,class_interval,quarter,timestamp,ef_day_y,fc_day_y
8865,A,6,25,9,1,198,A,7,3,15,...,4.0,0.0,0.0,0.0,0.0,0.0,3,1.625324e+09,184,176
8866,A,6,25,9,1,201,A,7,3,18,...,8.0,2.0,0.0,0.0,0.0,0.0,3,1.625335e+09,184,176
8867,A,6,25,9,1,204,A,7,3,21,...,8.0,2.0,0.0,0.0,0.0,0.0,3,1.625346e+09,184,176
8868,A,6,25,9,1,207,A,7,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,1.625357e+09,185,176
8869,A,6,25,9,1,210,A,7,4,3,...,9.0,5.0,1.0,0.0,0.0,0.0,3,1.625368e+09,185,176
8870,A,6,25,9,1,213,A,7,4,6,...,15.0,10.0,6.0,4.0,0.0,0.0,3,1.625378e+09,185,176
8871,A,6,25,9,1,216,A,7,4,9,...,15.0,10.0,6.0,4.0,0.0,0.0,3,1.625389e+09,185,176
8872,A,6,25,9,1,219,A,7,4,12,...,6.0,5.0,2.0,0.0,0.0,0.0,3,1.625400e+09,185,176
8873,A,6,25,9,1,222,A,7,4,15,...,6.0,5.0,2.0,0.0,0.0,0.0,3,1.625411e+09,185,176
8874,A,6,25,9,1,225,A,7,4,18,...,17.0,6.0,1.0,1.0,0.0,0.0,3,1.625422e+09,185,176


In [10]:
### 구간별 강수확률
col = [f'v0{i}' for i in range(1,10)]
temp = df_t[col]
temp2 = temp.shift(1, axis=1) - temp

# merge
temp = pd.concat([temp, temp2.iloc[:, 1:]], axis=1)
temp = temp.iloc[:, 8:]
temp.columns = [9] + list(range(1,8+1))

### 변수 추가하기
temp2 = temp[list(range(1,9+1))]
temp2.columns = [f'rng_v0{i}' for i in range(1,10)]
df_t = pd.concat([df_t, temp2], axis=1)

# 확인
df_t.iloc[8865:8865+10]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,fc_day_y,rng_v01,rng_v02,rng_v03,rng_v04,rng_v05,rng_v06,rng_v07,rng_v08,rng_v09
8865,A,6,25,9,1,198,A,7,3,15,...,176,9.0,12.0,6.0,2.0,8.0,4.0,0.0,0.0,0.0
8866,A,6,25,9,1,201,A,7,3,18,...,176,7.0,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0
8867,A,6,25,9,1,204,A,7,3,21,...,176,7.0,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0
8868,A,6,25,9,1,207,A,7,4,0,...,176,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
8869,A,6,25,9,1,210,A,7,4,3,...,176,3.0,1.0,3.0,3.0,5.0,4.0,4.0,1.0,0.0
8870,A,6,25,9,1,213,A,7,4,6,...,176,2.0,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0
8871,A,6,25,9,1,216,A,7,4,9,...,176,2.0,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0
8872,A,6,25,9,1,219,A,7,4,12,...,176,8.0,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0
8873,A,6,25,9,1,222,A,7,4,15,...,176,8.0,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0
8874,A,6,25,9,1,225,A,7,4,18,...,176,10.0,7.0,5.0,7.0,11.0,11.0,5.0,0.0,1.0


In [11]:
### pred
def max_find(row):
    # 행의 합이 0이면 0 반환
    if row.sum() == 0:
        return 0
    
    # 최대값 찾기
    max_value = max(row)

    # 최대값의 인덱스 찾기
    max_indices = [index for index, value in enumerate(row) if value == max_value]
    a = max(max_indices)

    return a

### 합치기
df_t['pred'] = temp.apply(max_find, axis=1)
df_t.iloc[8865:8865+10]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,rng_v01,rng_v02,rng_v03,rng_v04,rng_v05,rng_v06,rng_v07,rng_v08,rng_v09,pred
8865,A,6,25,9,1,198,A,7,3,15,...,9.0,12.0,6.0,2.0,8.0,4.0,0.0,0.0,0.0,2
8866,A,6,25,9,1,201,A,7,3,18,...,7.0,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0,2
8867,A,6,25,9,1,204,A,7,3,21,...,7.0,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0,2
8868,A,6,25,9,1,207,A,7,4,0,...,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2
8869,A,6,25,9,1,210,A,7,4,3,...,3.0,1.0,3.0,3.0,5.0,4.0,4.0,1.0,0.0,5
8870,A,6,25,9,1,213,A,7,4,6,...,2.0,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0,2
8871,A,6,25,9,1,216,A,7,4,9,...,2.0,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0,2
8872,A,6,25,9,1,219,A,7,4,12,...,8.0,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0,5
8873,A,6,25,9,1,222,A,7,4,15,...,8.0,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0,5
8874,A,6,25,9,1,225,A,7,4,18,...,10.0,7.0,5.0,7.0,11.0,11.0,5.0,0.0,1.0,6


In [ ]:
### -999대체
col = [f'v0{i}' for i in range(1,10)]
idx = df_t[df_t['class_interval'] == -999].index

df_t.loc[idx, 'class_interval'] = df_t.loc[idx, 'pred']
df_t.loc[idx, 'class_interval']

In [18]:
### pred_YN
# 0:No / 1:Yes

df_t['pred_YN'] = df_t.apply(lambda row: 1 if row['class_interval'] <= row['pred'] else 0, axis=1)

df_t.iloc[8865:8865+10]

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,rng_v02,rng_v03,rng_v04,rng_v05,rng_v06,rng_v07,rng_v08,rng_v09,pred,pred_YN
8865,A,6,25,9,1,198,A,7,3,15,...,12.0,6.0,2.0,8.0,4.0,0.0,0.0,0.0,2,1
8866,A,6,25,9,1,201,A,7,3,18,...,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0,2,1
8867,A,6,25,9,1,204,A,7,3,21,...,8.0,5.0,7.0,3.0,6.0,2.0,0.0,0.0,2,1
8868,A,6,25,9,1,207,A,7,4,0,...,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2,1
8869,A,6,25,9,1,210,A,7,4,3,...,1.0,3.0,3.0,5.0,4.0,4.0,1.0,0.0,5,1
8870,A,6,25,9,1,213,A,7,4,6,...,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0,2,1
8871,A,6,25,9,1,216,A,7,4,9,...,8.0,6.0,7.0,6.0,5.0,4.0,2.0,4.0,2,1
8872,A,6,25,9,1,219,A,7,4,12,...,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0,5,1
8873,A,6,25,9,1,222,A,7,4,15,...,13.0,8.0,10.0,15.0,1.0,3.0,2.0,0.0,5,1
8874,A,6,25,9,1,225,A,7,4,18,...,7.0,5.0,7.0,11.0,11.0,5.0,0.0,1.0,6,1


In [20]:
df_t.to_csv('./data/rainfall_train_prep.csv', index=False)
pd.read_csv('./data/rainfall_train_prep.csv')

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,rng_v02,rng_v03,rng_v04,rng_v05,rng_v06,rng_v07,rng_v08,rng_v09,pred,pred_YN
0,A,5,1,9,1,3,A,5,1,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,A,5,1,9,1,6,A,5,1,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,A,5,1,9,1,9,A,5,1,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,A,5,1,9,1,12,A,5,1,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,A,5,1,9,1,15,A,5,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457427,C,9,30,21,20,228,C,10,10,9,...,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0
1457428,C,9,30,21,20,231,C,10,10,12,...,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2,1
1457429,C,9,30,21,20,234,C,10,10,15,...,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2,0
1457430,C,9,30,21,20,237,C,10,10,18,...,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,5,1


In [18]:
### shift
col = [f'v0{i}' for i in range(1,10)] + [f'rng_v0{i}' for i in range(1,10)]

# drop index list
idx = df_t[df_t['dh'] == 3].index
t_idx = list(idx[:])

for val in col:
    temp = pd.DataFrame()
    temp['x'] = df_t[[val]]
    temp['y'] = df_t[['class_interval']]
    num, idx3 = spearmanr(temp['x'], temp['y'])
    
    for i in range(1, 14+1):
        temp['x'] = df_t[[val]].shift(i)

        temp.drop(index = t_idx, inplace=True)
        t_idx = list(idx+i)
        
        correlation, p_value = spearmanr(temp['x'], temp['y'])
        
        if p_value <= 0.05:
            if abs(correlation) >= num:
                num = correlation
                idx3 = i
        
    # 상관계수
    if idx3 > 0:
        print(val, ' : ----------------------------------------------------------------')
        print(f'i : {idx3} / corr : {num}')
        
print('ᐕ)ﾉ🎉') # done

ᐕ)ﾉ🎉


In [19]:
### roll
col = [f'v0{i}' for i in range(1,10)] + [f'rng_v0{i}' for i in range(1,10)]
result = {}

# drop index list
idx = df_t[df_t['dh'] == 3].index
t_idx = list(idx[:])

for val in col:
    temp = pd.DataFrame()
    temp['x'] = df_t[[val]]
    temp['y'] = df_t[['class_interval']]
    num, idx3 = spearmanr(temp['x'], temp['y'])
    
    for i in range(1, 14+1):
        temp['x'] = df_t[[val]].rolling(i, min_periods=1).mean()

        temp.drop(index = t_idx, inplace=True)
        t_idx = list(idx+i)
        
        correlation, p_value = spearmanr(temp['x'], temp['y'])
        
        if p_value <= 0.05:
            if abs(correlation) >= num:
                num = correlation
                idx3 = i
        
    # 상관계수
    if idx3 > 0:
        print(val, ' : ----------------------------------------------------------------')
        print(f'i : {idx3} / corr : {num}')
        result.update({val:idx3})
        
print('ᐕ)ﾉ🎉') # done

v07  : ----------------------------------------------------------------
i : 3 / corr : 0.352262051529024
v08  : ----------------------------------------------------------------
i : 4 / corr : 0.27796123469324024
v09  : ----------------------------------------------------------------
i : 5 / corr : 0.2138985264363226
rng_v01  : ----------------------------------------------------------------
i : 4 / corr : 0.13232432552145607
rng_v02  : ----------------------------------------------------------------
i : 3 / corr : 0.22156967519847232
rng_v03  : ----------------------------------------------------------------
i : 3 / corr : 0.31359494606967214
rng_v04  : ----------------------------------------------------------------
i : 2 / corr : 0.3790326575983584
rng_v05  : ----------------------------------------------------------------
i : 2 / corr : 0.4120047649290041
rng_v06  : ----------------------------------------------------------------
i : 2 / corr : 0.3917677108627269
rng_v07  : --------

In [20]:
# 변수 생성
for key, val in result.items():
    df_t[f'roll_{key}'] = df_t[[key]].rolling(val, min_periods=1).mean()
    
# drop
idx = df_t[df_t['dh'] == 3].index
t_idx = list(idx[:])

for i in range(1, 5+1):
    t_idx += list(idx+i)
df_t.drop(index = t_idx, inplace=True)

# 확인
display(df_t.iloc[8865:8865+10])
print(df_t.shape)

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,roll_v09,roll_rng_v01,roll_rng_v02,roll_rng_v03,roll_rng_v04,roll_rng_v05,roll_rng_v06,roll_rng_v07,roll_rng_v08,roll_rng_v09
9585,A,6,29,21,1,198,A,7,8,3,...,0.2,5.00,12.000000,12.333333,8.0,12.0,5.0,0.666667,0.50,0.2
9586,A,6,29,21,1,201,A,7,8,6,...,0.0,6.75,13.000000,12.333333,9.5,11.5,3.0,1.000000,0.25,0.0
9587,A,6,29,21,1,204,A,7,8,9,...,0.0,8.50,13.000000,12.666667,11.0,11.0,1.0,2.000000,0.00,0.0
9588,A,6,29,21,1,207,A,7,8,12,...,0.0,9.00,15.333333,14.000000,12.5,12.5,2.0,2.000000,0.00,0.0
9589,A,6,29,21,1,210,A,7,8,15,...,0.0,9.50,17.666667,15.000000,14.0,14.0,3.0,1.000000,0.00,0.0
9590,A,6,29,21,1,213,A,7,8,18,...,0.0,8.25,17.666667,14.000000,12.5,15.5,4.5,0.333333,0.00,0.0
9591,A,6,29,21,1,216,A,7,8,21,...,0.0,7.00,15.333333,12.000000,11.0,17.0,6.0,0.666667,0.00,0.0
9592,A,6,29,21,1,219,A,7,9,0,...,0.2,7.00,11.666667,11.000000,13.5,10.5,5.5,1.333333,0.25,0.2
9593,A,6,29,21,1,222,A,7,9,3,...,0.4,7.00,10.333333,12.000000,16.0,4.0,5.0,1.666667,0.50,0.4
9594,A,6,29,21,1,225,A,7,9,6,...,0.4,7.00,9.666667,12.000000,14.0,8.5,4.5,2.333333,1.25,0.4


(1347998, 48)


# 최종

In [ ]:
df_t = df_t.drop(columns=['fc_year', 'ef_year'])
df_t.to_csv('./data/rainfall_train_v1.csv', index=False)